In [11]:
import os
from pprint import pprint
from datetime import timedelta as delta, datetime
import pandas as pd
import numpy as np
import qgrid
import plotly.offline as offline
import plotly.tools as tools, plotly.graph_objs as go
from docs.config import *
from docs.rules import STRATS as rules
import app
from app import freqtostr, strtofreq
from app.bot import scanner, candles, signals

# Conf
pd.set_option("display.max_columns", 25)
pd.set_option("display.width", 2000)
app.set_db(["localhost", "45.79.176.125"][0])
db = app.get_db()

pair='POABTC'
freqstr = '5m'
pdfreqstr = '5T'
periods = 288
unit = 'minutes'
td = delta(minutes = 5 * periods)
freq = strtofreq[freqstr]

candles.update([pair], freqstr,
    start = "{} {} ago utc".format(periods * 3, unit),
    force=True)
app.bot.dfc = candles.merge_new(pd.DataFrame(), [pair],
    span = td * 3)

macd = signals.macd(app.bot.dfc.loc[pair, freq],
    rules['macd']['fast_span'],
    rules['macd']['slow_span']
).tail(periods)
scan = scanner.macd_analysis(pair, freqstr, periods, pdfreqstr=pdfreqstr)

# Build traces
fig = tools.make_subplots(
    rows=4,
    cols=1,
    specs=[ [{}], [{}], [{}], [{}] ], 
    shared_xaxes=False,
    shared_yaxes=True,
    vertical_spacing=0.005
)
fig.append_trace(
    go.Scatter(
        x=macd.index,
        y=macd['close'],
        name="Price"
    ),
    1, 1
)
fig.append_trace(
    go.Bar(
        x=macd.index,
        y=macd['macd_diff'],
        name="MACD_diff (normalized)"
    ),
    2, 1
)

# Layout
fig['layout'].update(
    title='{} MACD'.format(pair))
fig['layout']['xaxis2'].update(
    title="<BR><BR><BR>" + "<BR>".join(scan['output'][1:]))

file = "{}_{}_{}.html".format(pair, freqstr, str(datetime.now().date()))
offline.plot(fig, filename=file)
os.system("mv {} charts/".format(file))

len(macd)=176
end of histos
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



0

In [52]:
len(macd)

172

In [9]:
str(datetime.now().date())

'2018-04-07'